In [ ]:
from datasets import load_dataset
import tqdm

data = load_dataset("stogian/mrt_fp", split="train")

batch_size = 16
results = []

# Loop through each sample in the dataset.
for i in tqdm.tqdm(range(0, len(data), batch_size), desc="Processing batches", unit="batch"):
	batch = data[i:i+batch_size]
	messages = []
	for j in range(len(batch["question"])):
		sample = {key: batch[key][j] for key in batch}
		messages.append({
			"role": "user",
			"content": [
				{"type": "image", "image": sample["image"]},
				{"type": "text", "text": sample["question"]},
			],
		})
	print(f"Processing batch indices {i} to {min(i+batch_size, len(data))} with {len(messages)} messages")
	print(messages)

Processing batches: 100%|██████████| 4/4 [00:00<00:00, 108.81batch/s]

Processing batch indices 0 to 16 with 16 messages
[{'role': 'user', 'content': [{'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=173x142 at 0x7F3161D14C20>}, {'type': 'text', 'text': 'How man candidates are there in the image?\n'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=173x142 at 0x7F3161D17890>}, {'type': 'text', 'text': 'How man candidates are there in the image?\n'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=173x142 at 0x7F3161D164E0>}, {'type': 'text', 'text': 'How man candidates are there in the image?\n'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=173x142 at 0x7F3161D17500>}, {'type': 'text', 'text': 'How man candidates are there in the image?\n'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile 

In [4]:
import pandas as pd

res = pd.read_csv('results_llama_full.csv')

def preprocess(example):
	example['pred_y'] = example['prediction'].split('.')[0]
	return example

res = res.apply(preprocess, axis=1)
res


# print(
#     f"Correct {res['pred_y'].eq(res['ground_truth']).sum()} out of {len(res)} | {res['pred_y'].eq(res['ground_truth']).sum() / len(res) * 100:.2f}%"
# )

# # # Count A in "prediction" and "answer"
# print(f"Count A in prediction: {res['pred_y'].eq('B').sum()}")

,prediction,ground_truth,pred_y
0,A. Option A is simply the original shape in a ...,A,A
1,A. . . . . . . . . . . . . . . . . . . . . . ....,C,A
2,A. Option A is simply the original shape in a ...,B,A
3,C. . . . . . . . . . . . . . . . . . . . . . ....,D,C
4,A. . . . . . . . . . . . . . . . . . . . . . ....,C,A
...,...,...,...
95,A. . . . . . . . . . . . . . . . . . . . . . ....,B,A
96,A. . . . . . . . . . . . . . . . . . . . . . ....,D,A
97,A. . . . . A. . . . . . . . . . . . . . . . . ...,A,A
98,C. . . . . . . . . . . . . . . . . . . . . . ....,A,C


In [27]:
relations["prediction"][985]

'1. A\n2. B\n3. A\n4. B\n5. A\n6. A\n7. A\n8. A\n9. A\n10. A\n11. A\n12. A\n13. A\n14. A\n15. A\n16. A\n17. A\n18. A\n19. A\n20. A\n21. A\n22. A\n23. A\n24. A\n25. A\n26. A\n27. A\n2'

In [ ]:
import pandas as pd
from PIL import Image
import os
import json
from IPython.display import display, HTML
import base64

with open("output/imagesfiveshotv3/metadata.jsonl") as f:
	metadata = f.readlines()
metadata = [json.loads(x.strip()) for x in metadata]
df = pd.DataFrame(metadata)
# Function to encode image to base64
def image_to_base64(image_path, width=100):
	with open(image_path, "rb") as f:
		img_data = f.read()
	img_base64 = base64.b64encode(img_data).decode("utf-8")
	return f'<img src="data:image/jpeg;base64,{img_base64}" width="{width}"/>'
# Apply to DataFrame
df["image"] = df["image_path"].apply(image_to_base64)
# Save to HTML
# df.to_html("dataframe_with_images.html", escape=False)
# Display in Jupyter Notebook (optional)
display(HTML(df.to_html(escape=False)))


FileNotFoundError: [Errno 2] No such file or directory: 'output/imagesfiveshotv3/metadata.jsonl'

In [ ]:
# read jsonl file and display it in csv format through pandas
import pandas as pd
import json
import csv
import sys
import ast
import os
from PIL import Image as PILImage
from datasets import Dataset, Features, Value, Image

file = "llm_scene_description_with_image.jsonl"

def read_jsonl(file):
	with open(file, 'r') as f:
		data = f.readlines()
	return data

def jsonl_to_csv(data):
	with open('llm_scene_description_with_image.csv', 'w', newline='') as f:
		writer = csv.writer(f)
		header_written = False
		for line in data:
			item = json.loads(line)
			if not header_written:
				writer.writerow(item.keys())
				header_written = True
			writer.writerow(item.values())
   
data = read_jsonl(file)
jsonl_to_csv(data)
df = pd.read_csv('llm_scene_description_with_image.csv')


def process_gen_details(x):
	try:
		# Convert string representation of dict to actual dict
		d = ast.literal_eval(x)
		# Convert dict to a plain text string representation
		return ", ".join([f"{k}: {v}" for k, v in d.items()])
	except Exception as e:
		return x

df['generation_details'] = df['generation_details'].apply(process_gen_details)

# Improved helper function to load an image and convert it to RGB

def load_image(path):
	if not os.path.exists(path):
		print(f"File not found: {path}")
		return None
	try:
		with PILImage.open(path) as img:
			return img.convert("RGB")
	except Exception as error:
		print(f"Error loading {path}: {error}")
		return None


# Create a Hugging Face Dataset from the edited dataframe, letting the Image feature load images from the file paths
dataset = Dataset.from_pandas(
	df,
	features=Features(
		{
			"prompt_model": Value("string"),
			"prompt": Value("string"),
			"task_type": Value("string"),
			"diffusion_model": Value("string"),
			"image_path": Image(),  # Use the image_path column as the image feature
			"generation_details": Value("string"),
		}
	),
)
dataset


Dataset({
    features: ['prompt_model', 'prompt', 'task_type', 'diffusion_model', 'image_path', 'generation_details'],
    num_rows: 10
})

In [26]:

dataset.push_to_hub("test_dataset", private=True)  # Replace 'your-username' with your actual Hugging Face username.

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/stogian/test_dataset/commit/a803433d97210f446a53af892d7fb86760447005', commit_message='Upload dataset', commit_description='', oid='a803433d97210f446a53af892d7fb86760447005', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/stogian/test_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='stogian/test_dataset'), pr_revision=None, pr_num=None)

In [ ]:
from datasets import Dataset, Image, Features, Value
import json
from PIL import Image as PILImage
import os
import glob


def generate_examples():
	for data in annotations:
		image_path = os.path.join("Spatial-MM/data", data["image_name"])
		try:
			image = PILImage.open(image_path).convert("RGB")  # Load image as RGB
		except Exception as e:
			print("Error loading image: ", image_path)
			continue
		yield {
			"image": image,
			"question": data["question"],
			"answer": data["answer"],
		}


	# Load JSON annotations from multiple files
annotations = []
json_files = glob.glob("Spatial-MM/data/Spatial_MM_CoT/*.json")
for json_file in json_files:
	with open(json_file, "r") as f:
		annotations.extend(json.load(f))

print(f"Loaded {len(annotations)} annotations")

# Define dataset features (adjust based on your JSON structure)
# features = Features(
# 	{
# 		"image": Image(),
# 		"question": Value("string"),
# 		"answer": Value("string"),
# 	}
# )

# # Create the dataset
# dataset = Dataset.from_generator(
# 	generate_examples,
# 	features=features,
# )

# dataset
	# dataset.push_to_hub("spatial_mm", private=True)  # Push to the Hub



Loaded 2211 annotations


In [ ]:
import shutil
import os

source_dirs = ['Spatial-MM/data/Spatial_MM_Obj', 'Spatial-MM/data/Spatial_MM_CoT']
target_dir = 'Spatial-MM/data/images'

# Create target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

for source in source_dirs:
	for filename in os.listdir(source):
		source_file = os.path.join(source, filename)
		if os.path.isfile(source_file):
			shutil.move(source_file, target_dir)

In [ ]:
image_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".gif"]

source_dirs = ["Spatial-MM/data/Spatial_MM_Obj", "Spatial-MM/data/Spatial_MM_CoT"]
target_dir = "Spatial-MM/data/image_dir"

# Create target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

for source in source_dirs:
	for filename in os.listdir(source):
		if os.path.splitext(filename)[1].lower() in image_extensions:
			shutil.move(os.path.join(source, filename), target_dir)


In [ ]:
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
image_files = [
	f for f in os.listdir('Spatial-MM/data/Spatial_MM_Obj')
	if os.path.splitext(f)[1].lower() in image_extensions
]
print(f"Number of images: {len(image_files)}")

Number of images: 5702


In [ ]:
from datasets import Dataset, Image, Features, Value
import json
from PIL import Image as PILImage
import os
import glob


def generate_examples():
	for data in annotations:
		image_path = os.path.join("data/egobrientbench", data["image"])
		try:
			image = PILImage.open(image_path).convert("RGB")  # Load image as RGB
		except Exception as e:
			print("Error loading image: ", image_path)
			continue
		yield {
			"image": image,
			"question": data["question"],
			"answer": data["label"],
		}

	# Load JSON annotations from multiple files


annotations = []
json_files = glob.glob("data/egobrientbench/benchmark.json")
for json_file in json_files:
	with open(json_file, "r") as f:
		annotations.extend(json.load(f))

print(f"Loaded {len(annotations)} annotations")

# Define dataset features (adjust based on your JSON structure)
features = Features(
	{
		"image": Image(),
		"question": Value("string"),
		"answer": Value("string"),
	}
)

# Create the dataset
dataset = Dataset.from_generator(
	generate_examples,
	features=features,
)

dataset
# dataset.push_to_hub("spatial_mm", private=True)  # Push to the Hub


Loaded 33460 annotations


Dataset({
    features: ['image', 'question', 'answer'],
    num_rows: 33460
})

In [3]:
dataset.push_to_hub("stogian/egoorientbench", private=True )  # Push to the Hub

Uploading the dataset shards:   0%|          | 0/9 [00:00<?, ?it/s]

Map:   0%|          | 0/3718 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/38 [00:00<?, ?ba/s]

Map:   0%|          | 0/3718 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/38 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/3717 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/38 [00:00<?, ?ba/s]

Map:   0%|          | 0/3717 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/38 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/stogian/egoorientbench/commit/555bae171477e4755b0193eed2cc9f6c42b4d6e5', commit_message='Upload dataset', commit_description='', oid='555bae171477e4755b0193eed2cc9f6c42b4d6e5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/stogian/egoorientbench', endpoint='https://huggingface.co', repo_type='dataset', repo_id='stogian/egoorientbench'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoTokenizer, AutoModel
from huggingface_hub import notebook_login

!pip install transformers


model_name = "bert-base-uncased"
model = AutoModel.from_pretrained(model_name, cache_dir="../bin/models/"+model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="../bin/models/"+model_name)
notebook_login()

In [1]:
import pandas as pd

results = pd.read_csv("results.csv")

results

,prediction,ground_truth,correct,raw_prediction,candidate_order
0,4,3,False,Candidate 4. ### Answer: Candidate 4. ### Expl...,"['mirror', 'similar', 'rotate', 'mirror']"
1,2,4,False,Candidate 2. ### Answer: Candidate 2. ### Expl...,"['mirror', 'mirror', 'similar', 'rotate']"
2,1,4,False,"Candidate 1, Candidate 2, Candidate 3, Candida...","['mirror', 'similar', 'mirror', 'rotate']"
3,1,3,False,"Candidate 1, Candidate 2, Candidate 3, Candida...","['mirror', 'mirror', 'rotate', 'similar']"
4,4,1,False,Candidate 4. ### Answer: Candidate 4. ### Expl...,"['rotate', 'mirror', 'mirror', 'similar']"
...,...,...,...,...,...
495,1,1,True,Candidate 1. ### Answer: Candidate 1. ### Expl...,"['rotate', 'similar', 'mirror', 'mirror']"
496,2,1,False,Candidate 2. ### Answer: Candidate 2. ### Expl...,"['rotate', 'mirror', 'similar', 'mirror']"
497,1,2,False,Candidate 1. ### Answer: Candidate 1. ### Expl...,"['mirror', 'rotate', 'similar', 'mirror']"
498,3,4,False,Candidate 3.ourgardenstate.com/ourgarden.com/o...,"['mirror', 'similar', 'mirror', 'rotate']"


In [3]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score

y_true = results["ground_truth"]
y_pred = results["prediction"]

accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1: {f1:.2f}")

f1 = f1_score(y_true, y_pred, average="weighted")
print(f"F1: {f1:.2f}")


Accuracy: 0.23
Precision: 0.22
Recall: 0.23
F1: 0.19
F1: 0.19
